Basado en:

https://github.com/CONABIO/antares3/blob/develop/madmex/management/commands/ingest_training_from_vector.py

Se tiene que correr primero:

https://github.com/CONABIO/antares3-sandbox/blob/master/notebooks/agriculture_madmex_app/3a_assign_labels_and_insert_in_db_for_agriculture.ipynb

In [1]:
import fiona
from fiona.crs import to_string
from pyproj import Proj

from madmex.models import Tag,TrainClassificationLabeledByApp
from madmex.util.spatial import feature_transform
import logging


In [2]:
logger = logging.getLogger(__name__)

In [19]:
scheme_interpreted = 'agricultura_completo'
training_set = 'agricultura_4_clases_sample_1000_with_class_added'
input_file = '/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi_mean-ndvi_min-ndvi_max/02-01-2020/40_-31/02-01-2020_clusters_40_-31_nclusters_4_sample_1000_with_class.shp'
train_interpreted=True
field_interpreted = 'class'

In [4]:
with fiona.open(input_file) as src:
    p = Proj(src.crs)
    if p.crs.is_geographic: # Here we assume that geographic coordinates are automatically 4326 (not quite true)
        fc = list(src)
    else:
        crs_str = to_string(src.crs)
        fc = [feature_transform(x, crs_out='+proj=longlat', crs_in=crs_str)
              for x in src]

In [5]:
fc[524]

{'type': 'Feature',
 'id': '524',
 'properties': OrderedDict([('features_i', 359898),
              ('preds', 3),
              ('class', '59')]),
 'geometry': {'type': 'Polygon',
  'coordinates': (((-96.99919863482253, 20.118700078170395),
    (-96.99910270768027, 20.118696917470327),
    (-96.99910940015248, 20.118515699297426),
    (-96.99872569214081, 20.11850305591007),
    (-96.99872903862654, 20.11841244684428),
    (-96.99834533094281, 20.118399802496896),
    (-96.9983486776804, 20.118309193448003),
    (-96.99844460453052, 20.11831235462335),
    (-96.99882831197581, 20.118324998718823),
    (-96.99882496555865, 20.11841560777961),
    (-96.99911274638188, 20.11842509022211),
    (-96.9991160926068, 20.118334481154196),
    (-96.99921201949275, 20.1183376418448),
    (-96.99940387327806, 20.118343963044193),
    (-96.99940721930625, 20.11825335397661),
    (-96.99950314614155, 20.118256514483022),
    (-96.99950649210118, 20.118165905420465),
    (-96.99960241887686, 20.11816

In [6]:
fc[524]['properties'][field_interpreted]

'59'

In [7]:
Tag.objects.filter(scheme=scheme_interpreted).first()

<Tag: Tag object (54)>

In [23]:
def train_class_labeled_by_app_obj_builder(x):
    """x is a tuple (ValidObject, feature)"""
    tag_dummy = Tag()
    if train_interpreted:
        if Tag.objects.filter(scheme=scheme_interpreted).first() is not None:
            try:
                tag_interpreted = Tag.objects.get(pk=x['properties'][field_interpreted], scheme=scheme_interpreted)
            except:
                tag_interpreted = Tag()
        else:
            logger.info('Couldnt find scheme_interpreted, you need to first run antares register_tag, even so will continue ingestion process')
            tag_interpreted = Tag()
    else:
        tag_interpreted = Tag()
            
    return tag_interpreted

    

In [24]:
train_class_obj_list = [train_class_labeled_by_app_obj_builder(x) for x in fc]

In [25]:
train_class_obj_list[0]

<Tag: Tag object (None)>

In [27]:
train_class_obj_list[524].id

59